In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/adult-income-dataset/adult.csv")

In [ ]:
data.head(15)

In [ ]:
data=data.replace('?',np.nan)

In [ ]:
data.drop(['fnlwgt'],axis=1,inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data['workclass'].value_counts()

In [ ]:
data['workclass'] = data['workclass'].fillna("Private")


In [ ]:
data['occupation'].value_counts()

In [ ]:
data['occupation']=data['occupation'].fillna("Prof-specialty")

In [ ]:
data['native-country'].value_counts()

In [ ]:
data['native-country']=data['native-country'].fillna('United-States')

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
data[data.select_dtypes(include=["object"]).columns]=data[data.select_dtypes(include=["object"]).columns].apply(le.fit_transform)

In [ ]:
data.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X=data.drop('income',axis=1)
Y=data['income']
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.2, stratify=Y, random_state=42
)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [ ]:
K_range=range(1,31)
cv_scores=[]
for k in K_range:
    knn=KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn,X_scaled,Y,cv=5,scoring='accuracy',n_jobs=-1)
    cv_scores.append(scores.mean())
    


In [ ]:
best_k = K_range[np.argmax(cv_scores)]
best_k

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(K_range, cv_scores, marker='o')
plt.title("KNN Cross-Validation Accuracy for Different k")
plt.xlabel("k (Number of Neighbors)")
plt.ylabel("Average Cross-Validation Accuracy")
plt.grid(True)
plt.show()



In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, Y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy:", accuracy_score(Y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(Y_test, y_pred))
print("\nClassification Report:\n", classification_report(Y_test, y_pred))
